# ValueInvestor
We are looking to establish a robust intelligent system that can aid investment activities, using past stock performance data. 

Our goal in this project is to use provided historical trading data to predict future stock price valuations on a daily, weekly, and monthly basis. We also provide recommendations on whether to BUY, HOLD or SELL each stock based on the predictions.

### Data Description


We are given trading data from 8 different countries in emerging markets that include the 2020 Q1-Q2-Q3-Q4 and 2021 Q1 stock prices. Each market's operating days varies based on the country of the company and the market the stocks are exchanged. We will only 2020 data and predict with 2021 Q1 data.

Each set of data contain the following columns:
- Date
- Price (The closing price of the stock on a day)
- Open (The opening price of the stock on a day)
- High (The highest price of the stock on a day)
- Low (The lowers price of the stock on a day)
- Volume (The number of shares that were traded on a day)

### Data Loading

In [2]:
# Import the necessary packages
import pandas as pd

In [7]:
df = pd.read_excel("2020Q1Q2Q3Q4-2021Q1.csv")
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2021-03-31 00:00:00,291.02,294.0,294.42,290.26,47.00M,-0.0076
1,2021-03-30 00:00:00,293.25,294.86,295.72,291.5,38.81M,-0.0002
2,2021-03-29 00:00:00,293.3,289.72,294.09,289.26,43.68M,0.0071
3,2021-03-26 00:00:00,291.22,288.63,292.75,288.32,56.07M,0.0159
4,2021-03-25 00:00:00,286.66,286.5,287.03,283.85,35.22M,0.0030


### Data Exploration

### Create Model

In [7]:
# Choose a model
# TODO: Autoregressive integrated moving average

In [4]:
# Split data into train and test sets

In [5]:
# Fit model on the train set

### Forecast Future Unknown Data

In [6]:
# Re-fit model on entire dataset

### Summary